# Preparation

## Install libraries

In [1]:
!pip install -q -U langchain langchain_openai langgraph google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following

## Environment settings

In [2]:
import os
from google.colab import userdata

os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

OPENAI_MODEL = "gpt-4-turbo-preview"

# Define each components

## Define the utility functions

In [3]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", system_prompt),
      MessagesPlaceholder(variable_name="messages"),
      MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
  )
  agent = create_openai_tools_agent(llm, tools, prompt)
  return AgentExecutor(agent=agent, tools=tools)

def create_supervisor(llm: ChatOpenAI, agents: list[str]):
  system_prompt = (
    "You are the supervisor over the following agents: {agents}."
    " You are responsible for assigning tasks to each agent as requested by the user."
    " Each agent executes tasks according to their roles and responds with their results and status."
    " Please review the information and answer with the name of the agent to which the task should be assigned next."
    " Answer 'FINISH' if you are satisfied that you have fulfilled the user's request."
  )

  options = ["FINISH"] + agents
  function_def = {
    "name": "supervisor",
    "description": "Select the next agent.",
    "parameters": {
      "type": "object",
      "properties": {
        "next": {
          "anyOf": [
            {"enum": options},
          ],
        }
      },
      "required": ["next"],
    },
  }

  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", system_prompt),
      MessagesPlaceholder(variable_name="messages"),
      (
        "system",
        "In light of the above conversation, please select one of the following options for which agent should be act or end next: {options}."
      ),
    ]
  ).partial(options=str(options), agents=", ".join(agents))

  return (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="supervisor")
    | JsonOutputFunctionsParser()
  )

## Define the Tools

In [4]:
from langchain.tools import tool
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.messages import HumanMessage, SystemMessage

@tool("researcher")
def researcher(query: str) -> str:
  """Research by SERP API"""
  serp_api = SerpAPIWrapper()
  return serp_api.run(query)

@tool("writer")
def writer(content: str) -> str:
  """Write a blog"""
  chat = ChatOpenAI()
  messages = [
  SystemMessage(
    content="You are a blog writer specializing in IT technology. You are responsible for writing blog posts based on the content given."
            " Articles should be in markdown format."
            " You should also make it easy for the reader to read by dividing the content into sections, using tables and figures, etc."
    ),
    HumanMessage(
      content=content
    ),
  ]
  response = chat(messages)
  return response.content

@tool("reviewer")
def reviewer(content: str) -> str:
  """Review a blog"""
  chat = ChatOpenAI()
  messages = [
  SystemMessage(
    content="You are a reviewer specializing in IT technical blogs. You are responsible for reviewing the content given and ensuring that all of the following are met:"
            " 1. Blog must be written in Japanese"
            " 2. Blog must be written in markdown format"
            " 3. Blog must be easy for the reader to read, e.g., divided into sections according to content, or illustrated with tables and figures."
            " 4. Blog must be written the description on the latest information."
            " If all of the above criteria are met, please respond with 'APPROVE'."
            " If any of the above criteria are not met, respond with 'REQUEST_CHANGES' and describe what changes are required to make it 'APPROVE'."
    ),
    HumanMessage(
      content=content
    ),
  ]
  response = chat(messages)
  return response.content

## Define the Agents

In [5]:
from langchain_core.runnables import Runnable

llm = ChatOpenAI(model=OPENAI_MODEL)

def researcher_agent() -> Runnable:
  prompt = (
    "You are a researcher who uses SERP API's search engine to find the most up-to-date and correct information."
  )
  return create_agent(llm, [researcher], prompt)

def writer_agent() -> Runnable:
  prompt = (
    "You are a blog writer specializing in IT technology."
  )
  return create_agent(llm, [writer], prompt)

def reviewer_agent() -> Runnable:
  prompt = (
    "You are a reviewer specializing in IT technical blogs."
  )
  return create_agent(llm, [writer], prompt)

## Define the Nodes

In [6]:
import operator
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage

RESEARCHER = "RESEARCHER"
WRITER = "WRITER"
REVIEWER = "REVIEWER"
SUPERVISOR = "SUPERVISOR"

agents = [RESEARCHER, WRITER, REVIEWER]

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]
  next: str

def researcher_node(state: AgentState) -> dict:
  result = researcher_agent().invoke(state)
  return {"messages": [HumanMessage(content=result["output"], name=RESEARCHER)]}

def writer_node(state: AgentState) -> dict:
  result = writer_agent().invoke(state)
  return {"messages": [HumanMessage(content=result["output"], name=WRITER)]}

def reviewer_node(state: AgentState) -> dict:
  result = reviewer_agent().invoke(state)
  return {"messages": [HumanMessage(content=result["output"], name=REVIEWER)]}

def supervisor_node(state: AgentState) -> Runnable:
  return create_supervisor(llm, agents)

## Define the Graph

In [7]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node(RESEARCHER, researcher_node)
workflow.add_node(WRITER, writer_node)
workflow.add_node(REVIEWER, reviewer_node)
workflow.add_node(SUPERVISOR, supervisor_node)

workflow.add_edge(RESEARCHER, SUPERVISOR)
workflow.add_edge(WRITER, SUPERVISOR)
workflow.add_edge(REVIEWER, SUPERVISOR)
workflow.add_conditional_edges(
  SUPERVISOR,
  lambda x: x["next"],
  {
    RESEARCHER: RESEARCHER,
    WRITER: WRITER,
    REVIEWER: REVIEWER,
    "FINISH": END
  }
)

workflow.set_entry_point(SUPERVISOR)

graph = workflow.compile()

# Run

In [10]:
prompt = (
  "Please research 'LangChain' and output your findings in blog format."
)

for s in graph.stream({"messages": [HumanMessage(content=prompt)]}):
  if "__end__" not in s:
    print(s)
    print("----")

{'SUPERVISOR': {'next': 'RESEARCHER'}}
----
{'RESEARCHER': {'messages': [HumanMessage(content="# Exploring LangChain: A Framework for Language Model Integration\n\nIn the ever-evolving world of artificial intelligence and language models, there's a new framework on the block that's making waves for developers and AI enthusiasts alike. LangChain, developed by Harrison Chase and initially released in October 2022, promises to simplify the creation of applications using large language models (LLMs). With its latest stable release 0.1.1 launched in January 2024, LangChain is quickly gaining popularity for its innovative approach to language model integration.\n\n## What is LangChain?\n\nLangChain is a framework written in Python and JavaScript, designed to bridge the gap between the theoretical capabilities of language models and practical applications. It offers a flexible set of abstractions and an extensive toolkit that enables developers to create context-aware applications that levera